# Add Agency and Agency Information to FMIS IIJA

In [1]:
import numpy as np
import pandas as pd
from siuba import *

from shared_utils import geography_utils
from dla_utils import _dla_utils

from calitp import to_snakecase

/opt/conda/lib/python3.10/site-packages/geopandas/_compat.py:112: UserWarning: The Shapely GEOS version (3.10.3-CAPI-1.16.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.


In [2]:
pd.set_option("display.max_columns", 100)
pd.set_option('display.max_colwidth', None)


In [3]:
GCS_FILE_PATH  = 'gs://calitp-analytics-data/data-analyses/dla/dla-iija'

## Read In Data

In [4]:
locodes = to_snakecase(pd.read_excel(f"gs://calitp-analytics-data/data-analyses/dla/e-76Obligated/locodes_updated7122021.xlsx"))

In [5]:
county = to_snakecase(pd.read_excel(f"{GCS_FILE_PATH}/Copy of County.xlsx",
                                   sheet_name='locode1',
                                   header=[0]))

In [6]:
proj = to_snakecase(pd.read_excel(f"{GCS_FILE_PATH}/CopyofFMIS_Projects_Universe_IIJA_Reporting_4.xls", 
                           # sheet_name='FMIS 5 Projects  ', header=[3]
                           sheet_name='IIJA-combined',
                           # sheet_name='FMIS 5 Projects  ',
                           ))

In [7]:
# proj2 = to_snakecase(pd.read_excel(f"{GCS_FILE_PATH}/Copy of FMIS_Projects_Universe_(IIJA_Reporting)_ (5).xls", 
#                            sheet_name='FMIS 5 Projects  ', header=[3]))

In [8]:
# number of entries in the locodes list
len(locodes)

1041

In [9]:
# number of entries in the county locode list
len(county)

1072

In [10]:
county.sample(4)

,agency_locode,agency_name,district,mpo,county
74,5075,San Jacinto,8.00,SCAG,Riverside County
1017,6472,California Land Stewardship Institute,4.00,NON-MPO,NaN
113,5114,Sonoma,4.00,MTC,Sonoma County
243,5244,Patterson,10.00,STANCOG,Stanislaus County


In [11]:
locodes.head()

,agency_locode,agency_name,district,county_name,rtpa_name,mpo_name,mpo_locode_fads,active_e76s______7_12_2021_
0,6302,Humboldt Bay Harbor Recreation & Conservation District,1,Humboldt County,Humboldt County Association of Governments,NON-MPO,NON-MPO,NaN
1,6330,Willow Creek Community Services District,1,Humboldt County,Humboldt County Association of Governments,NON-MPO,NON-MPO,NaN
2,5036,Trinidad,1,Humboldt County,Humboldt County Association of Governments,NON-MPO,NON-MPO,NaN
3,5049,Ukiah,1,Mendocino County,Mendocino Council of Governments,NON-MPO,NON-MPO,Yes
4,5082,Willits,1,Mendocino County,Mendocino Council of Governments,NON-MPO,NON-MPO,NaN


In [12]:
proj.drop(columns =['unnamed:_0', 'unnamed:_13', 'total'], axis=1, inplace=True)

In [13]:
proj.sample()

,fmis_transaction_date,program_code,program_code_description,project_number,recipient_project_number,project_title,county_code,congressional_district,improvement_type,improvement_type_description,obligations_amount,summary_recipient_defined_text_field_1_value
248,2022-05-06,Y001//Y001,NATIONAL HIGHWAY PERF IIJA,5938176,10957015L,RIVER ROAD OVER SAN JOAQUIN RIVER (BRIDGE 39C0001) SEISMIC RETROFIT (REPLACEMENT),99,Cong Dist 10,15//16,Preliminary Engineering//Right of Way,"740,111.00",L5938STANCOG


## Get Locode Substring

In [14]:
string = proj['summary_recipient_defined_text_field_1_value'].iloc[0]

In [15]:
string

'L5253SCAG'

In [16]:
print(string.find('5'))
print(string.find('3'))

1
4


In [17]:
# need to extract string from position 1-4. 

In [18]:
proj['locode'] = proj.summary_recipient_defined_text_field_1_value.apply(lambda x: x[1:5])

In [19]:
proj.head()

,fmis_transaction_date,program_code,program_code_description,project_number,recipient_project_number,project_title,county_code,congressional_district,improvement_type,improvement_type_description,obligations_amount,summary_recipient_defined_text_field_1_value,locode
0,2022-06-01,YS30//YS30,HIGHWAY SAFETY IMP PROG IIJA,5253021,0720000168L,120TH STREET FROM PRAIRIE AVENUE TO FELTON AVENUE. TRAFFIC SIGNAL UPGRADES AT NINE SIGNALIZED INTERSECTIONS AND IMPROVE CROSSINGS AND SIGNAGE.,37,Cong Dist 43,17//21,Construction Engineering//Safety,"1,305,800.00",L5253SCAG,5253
1,2022-09-15,Y001,NATIONAL HIGHWAY PERF IIJA,NBIL539,0815000215L,"13TH STREET OVER WILSON CREEK FROM OAK GLEN ROAD TO KENTUCKY STREET, LWC 00L0017 REPLACE LOW WATER CROSSING WITH 2-LANE BRIDGE",71,Cong Dist 8,44,Other,"301,838.32",L5457SCAG,5457
2,2022-07-25,YS30//YS30,HIGHWAY SAFETY IMP PROG IIJA,5132050,0421000374L,"16 LOCATIONS IN THE CITY OF FAIRFIELD INSTALL ADAPTIVE SIGNAL TIMING AND ADVANCED DILEMMA-ZONE DETECTION, AND ENHANCE THE VISIBILITY OF SIGNALS.",95,Cong Dist 3,17//21,Construction Engineering//Safety,"935,150.00",L5132MTC,5132
3,2022-03-07,Y233//Y233//Y233//Y233,STBG IIJA OFF-SYSTEM BRIDGE,5914078,0100020461L,2.2 MILES NORTH OF STATE ROUTE 20 ON WITTER SPRINGS ROAD OVER COOPER CREEK. BR.# 14C0119 BRIDGE REPLACEMENT (TC),33,Cong Dist 3,11//15//16//17,Bridge Replacement - No Added Capacity//Preliminary Engineering//Right of Way//Construction Engineering,"1,615,000.00",L5914NON-MPO,5914
4,2022-07-18,YS30//YS30,HIGHWAY SAFETY IMP PROG IIJA,5094071,0419000558L,21 INTERSECTIONS; ON ALAMO DRIVE FROM MERCHANT STREET TO NUT TREE ROAD AND ON PEABODY ROAD FROM ELMIRA ROAD TO FOXBORO PARKWAY IMPROVE SIGNAL HARDWAR,95,Cong Dist 3,17//21,Construction Engineering//Safety,"2,027,700.00",L5094MTC,5094


In [20]:
county.sample()

,agency_locode,agency_name,district,mpo,county
939,6392,Family Service Agency of the Central Coast,5.00,AMBAG,Santa Cruz County


## Merge data on Locode

In [21]:
proj['locode'] = pd.to_numeric(proj['locode'], errors='coerce')

In [22]:
proj_all = (pd.merge(proj, locodes, left_on='locode', right_on='agency_locode', how='left'))

In [23]:
proj_all.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 330 entries, 0 to 329
Data columns (total 21 columns):
 #   Column                                        Non-Null Count  Dtype         
---  ------                                        --------------  -----         
 0   fmis_transaction_date                         330 non-null    datetime64[ns]
 1   program_code                                  330 non-null    object        
 2   program_code_description                      330 non-null    object        
 3   project_number                                330 non-null    object        
 4   recipient_project_number                      330 non-null    object        
 5   project_title                                 330 non-null    object        
 6   county_code                                   330 non-null    int64         
 7   congressional_district                        330 non-null    object        
 8   improvement_type                              330 non-null    object  

In [24]:
proj_all = proj_all.rename(columns={'agency_name':'implementing_agency',
                                   'locode':'implementing_agency_locode'})

In [25]:
#if we use other locode list then drop these columns
proj_all.drop(columns =['active_e76s______7_12_2021_', 'mpo_locode_fads', 'agency_locode'], axis=1, inplace=True)

In [26]:
proj_all.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 330 entries, 0 to 329
Data columns (total 18 columns):
 #   Column                                        Non-Null Count  Dtype         
---  ------                                        --------------  -----         
 0   fmis_transaction_date                         330 non-null    datetime64[ns]
 1   program_code                                  330 non-null    object        
 2   program_code_description                      330 non-null    object        
 3   project_number                                330 non-null    object        
 4   recipient_project_number                      330 non-null    object        
 5   project_title                                 330 non-null    object        
 6   county_code                                   330 non-null    int64         
 7   congressional_district                        330 non-null    object        
 8   improvement_type                              330 non-null    object  

In [27]:
# one locode did not match
proj_all>>filter(_.implementing_agency_locode.isnull())

,fmis_transaction_date,program_code,program_code_description,project_number,recipient_project_number,project_title,county_code,congressional_district,improvement_type,improvement_type_description,obligations_amount,summary_recipient_defined_text_field_1_value,implementing_agency_locode,implementing_agency,district,county_name,rtpa_name,mpo_name
284,2022-04-26,Y001,NATIONAL HIGHWAY PERF IIJA,NBIS522,0000001453L5,STATEWIDE - IN SERVICE BRIDGES OWNED BY LOCAL AGENCIES FEDERALLY MANDATED BRIDGE INSPECTION PROGRAM,67,Cong Dist 3,49,Bridge Inspection and Bridge Related Training,"14,164,800.00",S NON-MPO,NaN,NaN,NaN,NaN,NaN,NaN


In [28]:
proj_all.head()

,fmis_transaction_date,program_code,program_code_description,project_number,recipient_project_number,project_title,county_code,congressional_district,improvement_type,improvement_type_description,obligations_amount,summary_recipient_defined_text_field_1_value,implementing_agency_locode,implementing_agency,district,county_name,rtpa_name,mpo_name
0,2022-06-01,YS30//YS30,HIGHWAY SAFETY IMP PROG IIJA,5253021,0720000168L,120TH STREET FROM PRAIRIE AVENUE TO FELTON AVENUE. TRAFFIC SIGNAL UPGRADES AT NINE SIGNALIZED INTERSECTIONS AND IMPROVE CROSSINGS AND SIGNAGE.,37,Cong Dist 43,17//21,Construction Engineering//Safety,"1,305,800.00",L5253SCAG,"5,253.00",Hawthorne,7.00,Los Angeles County,Los Angeles County Metropolitan Transportation Auth.,Southern California Association Of Governments
1,2022-09-15,Y001,NATIONAL HIGHWAY PERF IIJA,NBIL539,0815000215L,"13TH STREET OVER WILSON CREEK FROM OAK GLEN ROAD TO KENTUCKY STREET, LWC 00L0017 REPLACE LOW WATER CROSSING WITH 2-LANE BRIDGE",71,Cong Dist 8,44,Other,"301,838.32",L5457SCAG,"5,457.00",Yucaipa,8.00,San Bernardino County,San Bernardino Associated Governments,Southern California Association Of Governments
2,2022-07-25,YS30//YS30,HIGHWAY SAFETY IMP PROG IIJA,5132050,0421000374L,"16 LOCATIONS IN THE CITY OF FAIRFIELD INSTALL ADAPTIVE SIGNAL TIMING AND ADVANCED DILEMMA-ZONE DETECTION, AND ENHANCE THE VISIBILITY OF SIGNALS.",95,Cong Dist 3,17//21,Construction Engineering//Safety,"935,150.00",L5132MTC,"5,132.00",Fairfield,4.00,Solano County,Metropolitan Transportation Commission,Metropolitan Transportation Commission
3,2022-03-07,Y233//Y233//Y233//Y233,STBG IIJA OFF-SYSTEM BRIDGE,5914078,0100020461L,2.2 MILES NORTH OF STATE ROUTE 20 ON WITTER SPRINGS ROAD OVER COOPER CREEK. BR.# 14C0119 BRIDGE REPLACEMENT (TC),33,Cong Dist 3,11//15//16//17,Bridge Replacement - No Added Capacity//Preliminary Engineering//Right of Way//Construction Engineering,"1,615,000.00",L5914NON-MPO,"5,914.00",Lake County,1.00,Lake County,Lake County/City Area Planning Council,NON-MPO
4,2022-07-18,YS30//YS30,HIGHWAY SAFETY IMP PROG IIJA,5094071,0419000558L,21 INTERSECTIONS; ON ALAMO DRIVE FROM MERCHANT STREET TO NUT TREE ROAD AND ON PEABODY ROAD FROM ELMIRA ROAD TO FOXBORO PARKWAY IMPROVE SIGNAL HARDWAR,95,Cong Dist 3,17//21,Construction Engineering//Safety,"2,027,700.00",L5094MTC,"5,094.00",Vacaville,4.00,Solano County,Metropolitan Transportation Commission,Metropolitan Transportation Commission


In [29]:
list(proj_all.implementing_agency.unique())

['Hawthorne',
 'Yucaipa',
 'Fairfield',
 'Lake County',
 'Vacaville',
 'Benicia',
 'Pico Rivera',
 'Marin County',
 'Los Banos',
 'Santa Clara County',
 'Santa Barbara County',
 'Fresno County',
 'Santa Ana',
 'Stockton',
 'Caltrans',
 'Sacramento',
 'San Benito County',
 'Pinole',
 'Sacramento County',
 'Merced County',
 'Shasta County',
 'Livingston',
 'El Dorado County',
 'Sanger',
 'Palmdale',
 'Tulare County',
 'Coachella',
 'Bakersfield',
 'Colton',
 'Visalia',
 'Rialto',
 'Metropolitan Transportation Commission',
 'Bellflower',
 'Belmont',
 'Larkspur',
 'Monterey County',
 'Madera County',
 'Moraga',
 'Orinda',
 'Redding',
 'Cathedral City',
 'Alameda',
 'Clovis',
 'Gardena',
 'Mission Viejo',
 'Pleasanton',
 'Fresno',
 'Butte County',
 'Rohnert Park',
 'Alameda County Transportation Commission',
 'Yolo County',
 'Stanislaus County',
 'Placer County',
 'Contra Costa County',
 'Mariposa County',
 'Santa Barbara',
 'Nevada County',
 'Calaveras County',
 'Long Beach',
 'Los Angeles

## Writing to GCS

In [30]:
#proj_all.to_csv(f"{GCS_FILE_PATH}/FMIS_projects_wip.csv")

## Adding Place Names (can do)
* we have a list of city names to county from [Caltrans PlaceNames](https://dot.ca.gov/-/media/dot-media/programs/research-innovation-system-information/documents/place-names/2019-place-names-in-california-a11y.pdf)


In [31]:
# city_place_names = (to_snakecase(pd.read_excel('gs://calitp-analytics-data/data-analyses/dla/e-76Obligated/2020-place-names-locode.xlsx', sheet_name=0)))

In [32]:
# city_place_names.sample()

In [33]:
# city_place_names.drop(columns =['unnamed:_1', 'unnamed:_3', 'unnamed:_4','unnamed:_6','unnamed:_7', 'date_of_incorporation',
#                                'city_name_abbr_','name','dist_', 'co_'], axis=1, inplace=True)

In [34]:
# (pd.merge(proj_all, city_place_names, left_on='agency_locode', right_on='ct_city_code', how='left', indicator=True))._merge.value_counts()

In [35]:
# proj_all1 = (pd.merge(proj_all, city_place_names, left_on='agency_locode', right_on='ct_city_code', how='left', indicator='City'))

In [36]:
# proj_all1.sample(2)

In [37]:
# proj_all1>>filter(_.City!='both')

In [38]:
# county_place_names = (to_snakecase(pd.read_excel('gs://calitp-analytics-data/data-analyses/dla/e-76Obligated/2020-place-names-locode.xlsx', sheet_name=1)))

In [39]:
# county_place_names

## Project Location

In [40]:
location = (proj_all>>select(_.implementing_agency, _.county_name, _.project_title))

In [41]:
location.sample()

,implementing_agency,county_name,project_title
214,Oakland,Alameda County,OAKPORT ST FROM EDGEWATER TO I-880 FREEWAY ENTRANCE; OAKLAND AVE FROM ORANGE STREET TO MACARTHUR; MONTEREY BLVD FROM MAIDEN LN TO BENNET PL IN OAKLAN


In [42]:
len(location)

330

## Project Title

In [43]:
#phrase extraction maybe: https://stackoverflow.com/questions/70995812/extract-keyword-from-sentences-in-a-pandas-text-column-using-nltk-and-or-regex

In [47]:
# ! pip install nltk
# ! pip install textblob
# ! pip install wordcloud

In [48]:
import nltk
from textblob import TextBlob
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize

In [49]:
import matplotlib.pyplot as plt
import matplotlib as mpl
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator

In [50]:

import re

In [51]:
#subset df to work on title
title = proj_all>>select(_.implementing_agency, _.program_code_description, _.project_title, _.program_code)

In [52]:
title

,implementing_agency,program_code_description,project_title,program_code
0,Hawthorne,HIGHWAY SAFETY IMP PROG IIJA,120TH STREET FROM PRAIRIE AVENUE TO FELTON AVENUE. TRAFFIC SIGNAL UPGRADES AT NINE SIGNALIZED INTERSECTIONS AND IMPROVE CROSSINGS AND SIGNAGE.,YS30//YS30
1,Yucaipa,NATIONAL HIGHWAY PERF IIJA,"13TH STREET OVER WILSON CREEK FROM OAK GLEN ROAD TO KENTUCKY STREET, LWC 00L0017 REPLACE LOW WATER CROSSING WITH 2-LANE BRIDGE",Y001
2,Fairfield,HIGHWAY SAFETY IMP PROG IIJA,"16 LOCATIONS IN THE CITY OF FAIRFIELD INSTALL ADAPTIVE SIGNAL TIMING AND ADVANCED DILEMMA-ZONE DETECTION, AND ENHANCE THE VISIBILITY OF SIGNALS.",YS30//YS30
3,Lake County,STBG IIJA OFF-SYSTEM BRIDGE,2.2 MILES NORTH OF STATE ROUTE 20 ON WITTER SPRINGS ROAD OVER COOPER CREEK. BR.# 14C0119 BRIDGE REPLACEMENT (TC),Y233//Y233//Y233//Y233
4,Vacaville,HIGHWAY SAFETY IMP PROG IIJA,21 INTERSECTIONS; ON ALAMO DRIVE FROM MERCHANT STREET TO NUT TREE ROAD AND ON PEABODY ROAD FROM ELMIRA ROAD TO FOXBORO PARKWAY IMPROVE SIGNAL HARDWAR,YS30//YS30
...,...,...,...,...
325,Indio,NATIONAL HIGHWAY PERF IIJA,"WESTBOUND INDIO BOULEVARD OVER WHITEWATER RIVER, BR. NO. 56C-0292 SEISMIC RETROFIT AND SCOUR COUNTERMEASURES",Y001//Y001
326,Kingsburg,CONGESTION MITIGATION IIJA,WESTSIDE OF 18TH AVE FROM STROUD AVE TO KLEPPER ST CONSTRUCT NEW SIDEWALKS,Y400//Y400
327,Santa Ana,TRANS ALTERNATIVES >200K IIJA,"WILLITS STREET FROM FAIRVIEW STREET TO E/S OF RAITT STREET INSTALL MEDIAN, PARKING-PROTECTED BICYCLE LANES, AND DEDICATED BICYCLE SIGNAL HEADS (TC)",Y301//Y301
328,San Joaquin County,STBG IIJA OFF-SYSTEM BRIDGE,WIMER ROAD OVER INDIAN CREEK NORTH BRANCH (BRIDGE 29C0303) BRIDGE REPLACEMENT (TC),Y233//Y233


In [53]:
title.program_code_description.value_counts()

HIGHWAY SAFETY IMP PROG IIJA      75
STBG-URBANIZED >200K IIJA         61
NATIONAL HIGHWAY PERF IIJA        59
STBG IIJA OFF-SYSTEM BRIDGE       49
CONGESTION MITIGATION IIJA        37
SURFAC TRNSP BLK GRTS-FLX IIJA    18
TRANS ALTERNATIVES >200K IIJA     14
PROJ TO REDUCE PM 2.5 EMI IIJA    12
TRANSP ALTERNATIVES FLEX IIJA      4
TRANS ALTERN 50K-200K POP IIJA     1
Name: program_code_description, dtype: int64

In [54]:
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/jovyan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [55]:
text = title[['project_title']]

In [56]:
title_text = text.squeeze()

In [57]:
title_list = title_text.tolist() 


In [58]:
title_list = ' '.join(title_list).lower()

In [59]:
title_list = re.sub(r'[^\w\s]','',title_list)

In [60]:
swords = [re.sub(r"[^A-z\s]", "", sword) for sword in stopwords.words('english')]


In [61]:
clean_title_list = [word for word in word_tokenize(title_list.lower()) if word not in swords] 

In [62]:
len(set(clean_title_list))


1729

In [63]:
#turn list back into df

In [64]:
clean_title_list_df = pd.DataFrame(np.array(clean_title_list))


In [65]:
clean_title_list_df.value_counts()

bridge     178
road       142
avenue     115
ave         77
street      75
          ... 
jamacha      1
bellota      1
belmont      1
j            1
00           1
Length: 1729, dtype: int64

In [66]:
clean_title_list_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4910 entries, 0 to 4909
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   0       4910 non-null   object
dtypes: object(1)
memory usage: 38.5+ KB


In [67]:
#splitting up the types of words based on the most common words appearing in title name

fix_type = ['REPLACEMENT', 'INSTALL', 'CONSTRUCT', 'REPLACE', 'SIGNAL', 'TRAFFIC',
           'IMPROVEMENT', 'PEDESTRIAN', 'LANES', 'NEW', 'REHABILITATION',
           'UPGRADE', 'CLASS', 'BIKE', 'WIDEN', 'LANDSCAPING', 'SAFETY', 'RAISED', 
            'SEISMIC', 'SIGNAGE', 'RETROFIT', 'ADD', 'PLANNING', 'PAVE'
           'PREVENTIVE','MAINTENANCE', 'REHAB', 'RESURFACE', 'REPAIR', 'ROUNDABOUT']

area_type = ['BRIDGE', 'ROAD', 'RD', 'AVENUE', 'AVE', 'STREET' , 'ST',
             'FRACTURED', 'LANE', 'DRIVE', 'boulevard', 'BLVD',
             'INTERSECTION', 'intersections', 'WAY', 'DR', 'CURB', 'ROADWAY',
             'TRAIL', 'PATH', 'CREEK', 'RIVER', 'SIDEWALK', 'CORRIDOR', 'PARKWAY',
            'RAMPS', 'GUARDRAIL']

jurisdiction = ['CITY', 'COUNTY', 'STATE', 'UNINCORPORATED']

other = ['TC', 'EXISTING']

In [68]:
def tokenize(texts):
    return [nltk.tokenize.word_tokenize(t) for t in texts]

### using  np.concatenate

In [69]:
#! pip install more-itertools

In [70]:
#from more_itertools import split_after

In [71]:
## code help: https://stackoverflow.com/questions/70995812/extract-keyword-from-sentences-in-a-pandas-text-column-using-nltk-and-or-regex
def key_word_intersection(df, text_col):
    summaries = []
    for x in tokenize(df[text_col].to_numpy()):
        keywords = np.concatenate([
                                np.intersect1d(x, ['BRIDGE REPLACEMENT', 'BRIDGE', 'INSTALL', 'CONSTRUCT', 'REPLACE',
                                                   'SIGNAL', 'SIGNALS', 'TRAFFIC', 'IMPROVEMENT', 'PEDESTRIAN', 
                                                   'LANES', 'NEW', 'REHABILITATION','UPGRADE', 'CLASS',
                                                   'BIKE', 'WIDEN', 'LANDSCAPING', 'SAFETY', 'RAISED', 
                                                   'SEISMIC', 'SIGNAGE', 'RETROFIT', 'ADD', 'PLANNING', 'PAVE',
                                                   'PREVENTIVE','MAINTENANCE', 'REHAB', 'RESURFACE', 'REPAIR', 'ROUNDABOUT'
                                                  'COMPLETE STREET', 'VIDEO DETECTION EQUIPMENT', 'SYNCHRONIZE CORRIDOR', 'ROADWAY REALIGNMENTS']),
                                np.intersect1d(x, [
                                    # 'BRIDGE', 'ROAD', 'RD', 'AVENUE', 'AVE', 'STREET' , 'ST',
                                                   # 'FRACTURED', 'LANE', 'DRIVE', 'BOULEVARD', 'BLVD',
                                                   'INTERSECTION', 'INTERSECTIONS', 'SIDEWALK', 
                                    # 'WAY', 'DR', 'CURB', 'ROADWAY',
                                                   # 'TRAIL', 'PATH', 'CREEK', 'RIVER', 
                                    # 'CORRIDOR', 'CROSSING','PARKWAY','RAMPS', 'GUARDRAIL'
                                ]), 
                                np.intersect1d(x, ['CITY', 'COUNTY', 'STATE', 'UNINCORPORATED'])])
    
        summaries.append(np.array(x)[[i for i, keyword in enumerate(x) if keyword in keywords]])
    return summaries 

In [72]:
text['summary'] = key_word_intersection(text, 'project_title')

/tmp/ipykernel_1931/4020948550.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [73]:
text.sample(10)

,project_title,summary
20,AMERICAN AVENUE FROM MADERA AVENUE TO PLACER AVENUE SHOULDER IMPROVEMENTS,[]
254,"SACRAMENTO ST FROM TENNESSEE ST TO CAPITAL ST IMPLEMENT ROAD DIET PROJECT . INSTALL NEW DESIGNATED CLASS 2 BIKE LANE OR BIKE SHARROWS, WITH CORRESP","[INSTALL, NEW, CLASS, BIKE, BIKE]"
40,"BELMONT: CHULA VISTA FROM ALAMEDA DE LAS PULGAS TO RALSTON AVE, 6TH AVENUE FROM RALSTON AVENUE TO HILL STREET, 6TH AVENUE FROM EMMETT AVENUE TO HARBO",[]
29,"AVE R BETWEEN SIERRA HIGHWAY AND 25TH STREET. AVE R COMPLETE STREET WITH SIDEWALKS GAP CLOSURES, BIKE LANES, ADA RAMPS AND ENHANCED CROSSWALKS","[BIKE, LANES]"
136,"IN ELK GROVE ON WATERMAN RD FROM 700 FT SOUTH OF BOND TO ELK GROVE BLVD, ON WATERMAN RD FROM CHAROLAIS WAY TO GRANT LINE RD, AND ON ELK GROVE FLORIN",[]
65,CONSTRUCT A MULTI-USE PATH (6 MILES) ALONG THE FRIANT-KERN CANAL IN BAKERSFIELD: BOUNDED BY 7TH STANDARD ROAD AND KERN RIVER PARKWAY. CONSTRUCT CLASS,"[CONSTRUCT, CONSTRUCT, CLASS]"
12,"ALDER CROFT HEIGHTS ROAD OVER HOOKER CREEK, 2.12 MILES SOUTH OF LOS GATOS SC ROAD. BRIDGE NO. 37C0506 REPLACE EXISTING ONE LANE BRIDGE WITH A NEW T","[BRIDGE, REPLACE, BRIDGE, NEW]"
238,"PORTIONS OF LAKE ISABELLA BOULEVARD AND ERSKINE CREEK ROAD CONSTRUCT BICYCLE AND PEDESTRIAN IMPROVEMENTS; BIKE BATH, BIKE LANES, CURB RAMPS AND SIDEW","[CONSTRUCT, PEDESTRIAN, BIKE, BIKE, LANES]"
43,BRIDGE 41C0032 ON AVENUE 25 OVER ASH SLOUGH REPLACE 2 LANE BRIDGE WITH 2 LANE BRIDGE,"[BRIDGE, REPLACE, BRIDGE, BRIDGE]"
27,"AT MISSOURI FLAT RD 0.8 MILES SOUTH OF GOLDEN CENTER DRIVE CONSTRUCT BIKE TRAIL CONNECTION, EXTEND CURB, GUTTER, AND SIDEWALK. TC","[CONSTRUCT, BIKE, SIDEWALK]"


In [74]:
text>>filter(_.project_title.str.contains('COMPLETE STREET'))

,project_title,summary
23,"ARDEN WAY FROM ETHAN WAY TO MORSE AVENUE. ARDEN WAY, FROM FULTON AVENUE TO MORSE AVENUE COMPLETE STREET IMPROVEMENTS INCLUDING SEPARATED SIDEWALK, C",[SIDEWALK]
29,"AVE R BETWEEN SIERRA HIGHWAY AND 25TH STREET. AVE R COMPLETE STREET WITH SIDEWALKS GAP CLOSURES, BIKE LANES, ADA RAMPS AND ENHANCED CROSSWALKS","[BIKE, LANES]"
99,EL CAMINO REAL (SR82) FROM ARROYO TO KAISER WAY COMPLETE STREET IMPROVEMENTS,[]
129,HEALDSBURG AVENUE BETWEEN POWELL AVENUE AND PASSALAQUA ROAD COMPLETE STREET IMPROVEMENTS,[]
290,THE TOWNSITE DRIVE COMPLETE STREETS PROJECT IS LOCATED IN THE CITY OF VISTA ALONG THE SEGMENT OF NORTH DRIVE FROM JUST NORTHWEST OF THE INTERSECTION,"[CITY, INTERSECTION]"


In [75]:

text>>filter(_.project_title.str.contains('BRIDGE REPLACEMENT'))

,project_title,summary
3,2.2 MILES NORTH OF STATE ROUTE 20 ON WITTER SPRINGS ROAD OVER COOPER CREEK. BR.# 14C0119 BRIDGE REPLACEMENT (TC),"[STATE, BRIDGE]"
6,3/8 MI WEST OF PARAMOUNT BLVDWASHINGTON BLVD OVER RIO HONDO.LOCAL BRIDGE # 53C0471 BRIDGE REPLACEMENT (DEMOLISH THE EXISTING 6 LANNE SUPERSTRUCTURE,"[BRIDGE, BRIDGE]"
10,ALAMITOS RD. BRIDGE @ ALAMITOS CK (37C0159) BRIDGE REPLACEMENT/SEISMIC RET (TC),"[BRIDGE, BRIDGE]"
24,ARROYA AVENUE OVER WEST DELTA CANAL (BRIDGE 39C0275) BRIDGE REPLACEMENT (TC),"[BRIDGE, BRIDGE]"
25,"ASH CREEK ROAD AT SACRAMENTO RIVER OVERFLOW, BRIDGE 06C0233 BRIDGE REPLACEMENT","[BRIDGE, BRIDGE]"
41,BON AIR BRIDGE (BON AIR RD OVER CORTE MADERA CREEK). BR.# 27C0028 BRIDGE REPLACEMENT,"[BRIDGE, BRIDGE]"
53,BUCK AVENUE OVER ALAMO CREEKBR NO. 23C0011 BRIDGE REPLACEMENT,[BRIDGE]
54,CANYON RD @ ACID CANAL. BR. # 06C0307 BRIDGE REPLACEMENT,[BRIDGE]
55,CASSEL FALL RIVER ROAD BRIDGE AT PIT RIVER. BR. # 06C0039 BRIDGE REPLACEMENT,"[BRIDGE, BRIDGE]"
68,"CR 40 OVER CACHE CREEK, 0.12 MILES SOUTH OF SR 16 BRIDGE REPLACEMENT. REPLACE EXISTING 1 LANE BRIDGE WITH A NEW 1 LANE BRIDGE. BR#22C0091 (TC)","[BRIDGE, REPLACE, BRIDGE, NEW, BRIDGE]"


In [76]:
text>>filter(_.project_title.str.contains('WIDEN'))

,project_title,summary
81,"DOWNTOWN LOS ANGELES, BROADWAY BETWEEN 4TH AND 6TH STREETS PEDESTRIAN SAFETY IMPROVEMENTS INCLUDING CURB EXTENSIONS, WIDENED SIDEWALK, CROSSWALK AND","[PEDESTRIAN, SAFETY, SIDEWALK]"
104,"FIRST STREET BETWEEN FLOWER ST AND STANDARD AVE WIDEN EXISTING SIDEWALKS BY 3FT, NARROW VEHICLE LANES, AND CONSTRUCT ADA IMPROVEMENTS ON SIDEWALKS AN","[WIDEN, LANES, CONSTRUCT]"
156,"IN STOCKTON, PARALLEL TO MARCH LANE IN THE EAST BAY MUD CORRIDOR BETWEEN BROOKSIDE ROAD AND HILLSBORO WAY. RECONSTRUCT, WIDEN AND IMPROVE EXISTING PA",[WIDEN]
158,"IN THE CITY OF GALT ON KOST ROAD AT UNION PACIFIC RAILROAD CROSSING, WEST OF JOY DR., EAST OF MARIA WAY. WIDEN 400 FEET OF KOST RD. ON EACH SIDE OF T","[CITY, WIDEN]"
181,LA PAZ ROAD: MUIRLANDS BLVD. TO CRISANTA DR. WIDEN TWO OVERHEADS OVER BNSF,[WIDEN]
194,MILLERTON ROAD FROM FRIANT ROAD TO MARINA DRIVE WIDEN ROADWAY FROM 2LU TO 4LD,[WIDEN]
204,"NEES AVENUE FROM MINNEWAWA AVENUE TO CLOVIS AVENUE ROAD WIDENING AND RECONSTRUCTION, INSTALLATION OF CURB AND GUTTERS, SIDEWALK, BICYCLE LANES, MODIF","[SIDEWALK, LANES]"
220,ON OLD OREGON TRAIL BETWEEN PASO ROBLES AVENUE AND BEAR MOUNTAIN ROAD WIDEN SHOULDERS AND UPGRADE DRAINAGE (TC),"[WIDEN, UPGRADE]"
233,PARK ROAD FROM 250 FT SOUTH OF OAK ROAD TO BAYSHORE ROAD REHAB./RESURFACE ROADWAY AND WIDEN ROAD TO INSTALL ASPHALT CONCRETE PAVED CLASS II/V BICYC,"[WIDEN, INSTALL, CLASS]"
241,"POWER INN ROAD FROM ELSIE AVENUE TO ABOUT 400 FEET NORTH OF MACFADDEN DRIVE. INSTALL CURB, GUTTER, SIDEWALK INFILL AND CURB RAMPS; WIDEN SUBSTANDARD","[INSTALL, SIDEWALK, WIDEN]"


In [77]:
type(text['summary'])

pandas.core.series.Series

### using if statement

In [78]:
title.sample()

,implementing_agency,program_code_description,project_title,program_code
83,Dublin,STBG-URBANIZED >200K IIJA,DUBLIN -- SEGMENT OF DUBLIN BLVD. FROM SCARLET DR. TO HACIENDA DRIVE. - REHABILITATE PAVEMENT.,Y230


In [79]:
type_list = ['BRIDGE REPLACEMENT', 'COMPLETE STREET', 'VIDEO DETECTION EQUIPMENT', 'SYNCHRONIZE CORRIDOR', 'ROADWAY REALIGNMENTS']

In [80]:
text>>filter(
            _.project_title.str.contains('UPGRADE')| _.project_title.str.contains('IMPROVE')
            )

,project_title,summary
0,120TH STREET FROM PRAIRIE AVENUE TO FELTON AVENUE. TRAFFIC SIGNAL UPGRADES AT NINE SIGNALIZED INTERSECTIONS AND IMPROVE CROSSINGS AND SIGNAGE.,"[TRAFFIC, SIGNAL, INTERSECTIONS, SIGNAGE]"
4,21 INTERSECTIONS; ON ALAMO DRIVE FROM MERCHANT STREET TO NUT TREE ROAD AND ON PEABODY ROAD FROM ELMIRA ROAD TO FOXBORO PARKWAY IMPROVE SIGNAL HARDWAR,"[INTERSECTIONS, SIGNAL]"
5,21 LOCATIONS IN CITY OF BENICIA UPGRADE EXISTING GUARDRAILS AND END TREATMENTS.,"[CITY, UPGRADE]"
20,AMERICAN AVENUE FROM MADERA AVENUE TO PLACER AVENUE SHOULDER IMPROVEMENTS,[]
22,"APPIAN WAY AND MARLESTA ROAD INTERSECTION SAFETY IMPROVEMENTS: INSTALLATION OF A TRAFFIC SIGNAL AND TRAFFIC SIGNAGE, INCLUDING ADVANCED WARNING SIGNS","[INTERSECTION, SAFETY, TRAFFIC, SIGNAL, TRAFFIC, SIGNAGE]"
23,"ARDEN WAY FROM ETHAN WAY TO MORSE AVENUE. ARDEN WAY, FROM FULTON AVENUE TO MORSE AVENUE COMPLETE STREET IMPROVEMENTS INCLUDING SEPARATED SIDEWALK, C",[SIDEWALK]
26,AT MAX FOSTER SPORTS COMPLEX AREA IN LIVINGSTON CONSTRUCT MULTI-USE PATH IMPROVEMENTS,[CONSTRUCT]
81,"DOWNTOWN LOS ANGELES, BROADWAY BETWEEN 4TH AND 6TH STREETS PEDESTRIAN SAFETY IMPROVEMENTS INCLUDING CURB EXTENSIONS, WIDENED SIDEWALK, CROSSWALK AND","[PEDESTRIAN, SAFETY, SIDEWALK]"
85,E. 20TH STREET FROM THE MALL TO THE END OF BUSINESS LANE (ADJACENT TO SR99) BIKEWAY IMPROVEMENTS,[]
99,EL CAMINO REAL (SR82) FROM ARROYO TO KAISER WAY COMPLETE STREET IMPROVEMENTS,[]


In [81]:
text.sample(10)

,project_title,summary
228,ORWOOD BRIDGE (28C-0024) AT INDIAN SLOUGH BRIDGE REPLACEMENT (TC),"[BRIDGE, BRIDGE]"
192,MICHIGAN AVE. BETWEEN 19TH & 20TH STREET BIKE PATH IMPROVEMENTS,[BIKE]
119,FREMONT BLVD. AND WALNUT AVE. INTERSECTION CONNECT TO EXISTING ELEVATED/SEPARATED CLASS IV BIKEWAYS AND SIDEWALKS,"[INTERSECTION, CLASS]"
143,IN ELK GROVE ON GRANT LINE ROAD FROM SHERMAN OAKS CT. TO ALEILANI LANE. INTERSECTION OPERATIONAL IMPROVEMENTS IN THE FORM OF EITHER SIGNAL MODIFICATI,"[INTERSECTION, SIGNAL]"
49,BRIDGE 46C0196 ON M375A MINERAL KING ROAD OVER EAST FORK KAWEAH RIVER REPLACE 2 LANE BRIDGE WITH 2 LANE BRIDGE (TC),"[BRIDGE, REPLACE, BRIDGE, BRIDGE]"
297,US-50 AT EMPIRE RANCH ROAD CONSTRUCT 4 LANE INTERCHANGE (TC),[CONSTRUCT]
166,INTERSECTION OF REDWOOD DRIVE AND REDWAY DRIVE IN THE TOWN OF REDWAY. INSTALL ENHANCED PEDESTRIAN CROSSING WITH BULB OUTS TO CROSS REDWAY DRIVE INSTA,"[INTERSECTION, INSTALL, PEDESTRIAN]"
126,"GOLDEN VALLEY PARKWAY (TOWNE CENTRE DRIVE AND LATHROP ROAD), RIVER ISLANDS PARKWAY (GOLDEN VALLEY PARKWAY LATHROP FIRE STATION 34), LATHROP ROAD. ROA",[]
157,"IN THE CITY OF FOLSOM. THIS IS AN ITS MASTER PLAN. THE PROJECT WILL DESIGN AND PROVIDE PLANS, SPECIFICATIONS AND ESTIMATES (PS&E) OF TRAFFIC SIGNALS","[CITY, TRAFFIC, SIGNALS]"
134,"I-10 CORRIDOR CONTRACT 2: THE PROJECT WILL PROVIDE ONE EXPRESS LANES IN EACH DIRECTION FROM JUST EAST OF I-15 TOPEPPER AVENUE IN COLTON, CONNECTING T",[LANES]


In [120]:
#  def project_cat(row): 
#         if (row.project_title.str.contains("BRIDGE REPLACEMENT")):
#             return "Bridge Replacement"
        
#         elif (row.project_title.str.contains(" WIDEN SHOULDERS ")):
#             return "Widen Shoulders"
        
#         elif (row.project_title.str.contains("SYNCHRONIZE CORRIDOR")):
#             return "Synchronize Corridor"
        
#         elif (row.project_title.str.contains("COMPLETE STREET")):
#             return "Complete Streets"
        
#         elif (row.project_title.str.contains('TRAFFIC SIGNAL') and row.project_title.str.contains('INSTALL')):
#             return "Install Traffic Signal"
        
#         elif (row.project_title.str.contains('BRIDGE PREVENTIVE MAINTENANCE')):
#             return "Bridge Preventive Maintenance"
        
#         elif (row.project_title.str.contains("UPGRADE") and row.project_title_str.contains('GUARDRAIL')):
#             return "Upgrade Guardrail"
#         elif (row.project_title.str.contains("UPGRADE") and row.project_title_str.contains('TRAFFIC SIG')):
#             return "Upgrade Traffic Signal"
    
#         ## return string to identify which cases do not fit.
#         else:
#             return ""
        
#     # df['project_type'] = df.apply(lambda x: project_cat(x), axis=1)

In [191]:
title>>filter(_.project_title.str.contains('REHAB'))

,implementing_agency,program_code_description,project_title,program_code,Activity_2,Activity_1
21,San Benito County,NATIONAL HIGHWAY PERF IIJA,ANZAR ROAD BRIDGE #43C0039 REHABILITATE BRIDGE,Y001,Project,
33,Bakersfield,STBG-URBANIZED >200K IIJA,BAKERSFIELD: CALIFORNIA AVE FROM UNION AVE TO WASHINGTON STREET PAVEMENT REHABILITATION,Y230//Y230,Project,
38,Sacramento,STBG-URBANIZED >200K IIJA,BELL AVE. FROM BOLLENBACHER AVE. TO ASTORIA ST ROAD REHABILITATION (TC),Y230//Y230,Project,
83,Dublin,STBG-URBANIZED >200K IIJA,DUBLIN -- SEGMENT OF DUBLIN BLVD. FROM SCARLET DR. TO HACIENDA DRIVE. - REHABILITATE PAVEMENT.,Y230,Project,
97,San Joaquin County,NATIONAL HIGHWAY PERF IIJA,EIGHT MILE ROAD OVER WHITE SLOUGH (HONKER CANAL) - (BRIDGE 29C0219) BRIDGE REHABILITATION,Y001//Y001,Project,
125,San Diego,STBG IIJA OFF-SYSTEM BRIDGE,GEORGIA ST OC UNIVERSITY AVE 57C-0418 BRIDGE SEISMIC RETROFIT & REHABILITATION (TC),Y233//Y233,Project,
131,Hercules,STBG-URBANIZED >200K IIJA,HERCULES: SYCAMORE AVE FROM CIVIC DR TO WILLOW/PALM AVE PAVEMENT REHABILITATION,Y230,Project,
137,Caltrans,CONGESTION MITIGATION IIJA,"IN AND NEAR GRIDLEY, FROM HOLLIS LANE TO NORTH OF FORD AVENUE. DIST-CO-RTE: 03-BUT-99 PM/PM: R3.1/5.0 REHABILITATE PAVEMENT, UPGRADE TRANSPORTATION M",Y400//Y400,Project,Upgrade
161,Sacramento,STBG-URBANIZED >200K IIJA,IN THE CITY OF SACRAMENTO - ON FLORIN ROAD FROM 24TH STREET TO MUNSON WAY PAVEMENT REHABILITATION ON FOUR TRAVEL LANES PLUS A RAISED MEDIAN AND CENTE,Y230,Project,
174,Newman,SURFAC TRNSP BLK GRTS-FLX IIJA,"INYO AVE FROM L ST TO CANAL SCHOOL RD; CANAL SCHOOL RD FROM CITY LIMITS TO HILLS FERRY RD. PAVEMENT REHABILITATION, PULVERIZE EXISTING PAVEMENT AND P",Y240,Project,


In [206]:
def add_description(df):
    ##using np.where. code help: https://stackoverflow.com/questions/43905930/conditional-if-statement-if-value-in-row-contains-string-set-another-column
    df['Activity_1'] = (np.where(df.project_title.str.contains("INSTALL"), "Install",
                        np.where(df.project_title.str.contains("CONSTRUCT"), "Construct",
                        np.where(df.project_title.str.contains("UPGRADE"), "Upgrade",
                        np.where(df.project_title.str.contains("IMPROVE"), "Improve",
                        np.where(df.project_title.str.contains("REPLACE"), "Replace",
                        np.where(df.project_title.str.contains("REPLACE")& df.project_title.str.contains("GUARDRAIL"), "Replace",
                                    "")))))))
    
    df['Activity_2'] = (np.where(df.project_title.str.contains("BRIDGE REPLACEMENT"), "Bridge Replacement",
                        np.where(df.project_title.str.contains("WIDEN SHOULDERS"), "Widen Shoulders",
                        np.where(df.project_title.str.contains("SYNCHRONIZE CORRIDOR"), "Synchronize Corridor",
                        np.where(df.project_title.str.contains("COMPLETE STREET"), "Complete Streets",
                        np.where(df.project_title.str.contains("BRIDGE PREVENTIVE MAINTENANCE"), "Bridge Preventive Maintenance",
                        np.where(df.project_title.str.contains("SIDEWALK"), "Sidewalk",
                        np.where(df.project_title.str.contains("SCOUR"), "Erosion Countermeasures",
                        np.where(df.project_title.str.contains("ROUNDABOUT"), "Roundabout",
                        np.where(df.project_title.str.contains("GUARDRAIL"), "Guardrails",
                        np.where(df.project_title.str.contains("VIDEO DETECTION EQUIPMENT"), "Video Detection Equipment",
                        np.where(df.project_title.str.contains("PEDESTRIAN") & df.project_title.str.contains("BIKE") , "Pedestrian  & Bike Safety Improvements",
                        np.where(df.project_title.str.contains("BRIDGE") & df.project_title.str.contains("REHAB") , "Bridge Rehabilitation",
                        np.where(df.project_title.str.contains("PAVEMENT") & df.project_title.str.contains("REHAB") , "Pavement Rehabilitation",
                        np.where(df.project_title.str.contains("PEDESTRIAN"), "Pedestrian Safety Improvements",
                        np.where(df.project_title.str.contains("TRAFFIC SIG"), "Traffic Signal",
                        np.where(df.project_title.str.contains("SIGNAL"), "Signal",
                        np.where(df.project_title.str.contains("BRIDGE"), "Bridge",
                                 'Project')
                                   )))))))))))))))))
    
    return df

In [207]:
list = ['TRAFFIC','LANES', 'NEW', 'REHABILITATION', 'CLASS',
 'BIKE', 'WIDEN', 'LANDSCAPING', 'SAFETY', 'RAISED', 
 'SEISMIC', 'SIGNAGE', 'RETROFIT', 'ADD', 'PLANNING', 'PAVE',
 'PREVENTIVE','MAINTENANCE', 'REHAB', 'RESURFACE', 'REPAIR', 'ROUNDABOUT'
 'COMPLETE STREET', 'VIDEO DETECTION EQUIPMENT', 'ROADWAY REALIGNMENTS','INTERSECTION', 'INTERSECTIONS']
              

In [208]:
(add_description(title))>>filter(_.Activity_2 == 'Project')

,implementing_agency,program_code_description,project_title,program_code,Activity_2,Activity_1
7,Marin County,STBG-URBANIZED >200K IIJA,"4 AT TERNERS DRIVE @ DRIVEWAY ENTRANCES TO MULTI-FAMILY HOUSING CLOSEST TO DONAHUE, 4 AT TERNERS DRIVE @ TERRACE WAY, 4 AT TERNERS DRIVE @ TERRACE DR",Y230,Project,
15,Fresno County,TRANS ALTERNATIVES >200K IIJA,"ALONG GROVE AVENUE FROM PROSPECT AVENUE TO VALENTINE AVENUE, AND ALONG VALENTINE AVENUE FROM GROVE AVENUE TO NORTH AVENUE INSTALL AN ASPHALT CONCRETE",Y301//Y301,Project,Install
16,Santa Ana,TRANS ALTERNATIVES >200K IIJA,ALONG NEIGHBORHOOD STREETS ADJACENT TO DAVIS ELEMENTARY SCHOOL FROM 14TH STREET TO 15TH STREET. CONSTRUCT BULB-OUT AT CROSSINGS NEAR DAVIS ELEMENTARY,Y301,Project,Construct
17,Stockton,HIGHWAY SAFETY IMP PROG IIJA,ALONG PACIFIC AVE. FROM CALAVERAS RIVER TRAIL TO WEST HAMMER LANE. INSTALL RAISED MEDIAN CURB,YS30,Project,Install
18,Caltrans,STBG-URBANIZED >200K IIJA,"ALONG THE I-880 CORRIDOR SF BAY AREA: REGIONWIDE: IMPLEMENT A COLLECTIVE APPROACH TO FREEWAY OPERATIONS AND MANAGEMENT, INCLUDING COMMUNICATIONS NETW",Y230,Project,
...,...,...,...,...,...,...
314,Windsor,SURFAC TRNSP BLK GRTS-FLX IIJA,VARIOUS LOCATIONS WITHIN THE CITY OF HEADSBURG. NON-INFRASTRUCTURE PROJECT TO SET UP AND OPERATE A 3 YEAR E-BIKE SHARE PILOT PROGRAM. WORK INCLUDES,Y240,Project,
316,Calaveras County,HIGHWAY SAFETY IMP PROG IIJA,VARIOUS ROAD SEGMENTS THROUGHOUT CALAVERAS COUNTY (EXCLUDING STATE ROUTES AND THE CITY OF ANGELS CAMP) CONDUCT A ROADWAY SAFETY SIGNAGE AUDIT (RSSA),YS30//YS30,Project,
318,Inglewood,HIGHWAY SAFETY IMP PROG IIJA,"VARIOUS UNCONTROLLED CROSSWALKS THROUGHOUT THE CITY OF INGLEWOOD, INCLUDING CENTINELA AVE AT MARKET ST, MANCHESTER BLVD AT CEDAR AVE, ON REGENT ST BE",YS30,Project,
320,Clovis,STBG-URBANIZED >200K IIJA,"VILLA AVENUE BETWEEN BULLARD AVENUE AND BARSTOW AVENUE ROAD REHABILITATION INCLUDING GRINDING, PAVING, CONCRETE, INSTALLING TRAFFIC DEVICES AND RESTR",Y230,Project,Install
